# Create virtual machines in Orbstack(MacOS)/ multipass(Ubuntu)

This setup is meant for running Ansible playbooks on MacOS/Ubuntu. 

## Prerequisites for MacOs
1. Install [OrbStack](orbstack.dev)
2. Confirm `orb` command is now available on your command line
3. Install [Ansible](https://docs.ansible.com/ansible/latest/installation_guide/intro_installation.html)

## Prerequisites for Ubuntu
1. Install [multipass](https://canonical.com/multipass/install)
2. Confirm installation with `multipass --version` command
3. Install [Ansible](https://docs.ansible.com/ansible/latest/installation_guide/installation_distros.html#installing-ansible-on-debian)

Either run the commands manually from command like (for live output) or by executing cells here

Identifying the OS you are currently using

In [ ]:
import platform

linux = platform.system() == "Linux"

Running the provision script according to OS

In [ ]:
import subprocess
if linux:
    subprocess.run(["bash", "./provision-linux.sh"], check=True)
else:
    subprocess.run(["bash", "./provision-mac.sh"], check=True)


Notice that the following task will prompt (in VSCode) for your dockerhub username and password

In [ ]:
import os

os.environ["DOCKER_USERNAME"] = input("Enter Dockerhub username: ")
os.environ["DOCKER_PASSWORD"] = input("Enter Dockerhub password: ")

In [117]:
import os
env_vars = {
  'ALERT_EMAIL': 'example@example.com',
  'MINIO_ROOT_USER': 'minioadmin',
  'MINIO_ROOT_PASSWORD': 'minioadmin',
  'MINIO_BUCKET': 'ocrvs',
  'ELASTICSEARCH_SUPERUSER_PASSWORD': 'changeme_es',
  'KIBANA_USERNAME': 'kibbe',
  'KIBANA_PASSWORD': 'changeme_kibana',
  'KIBANA_SYSTEM_PASSWORD': 'changeme_kibana_system',
  'MONGODB_ADMIN_USER': 'admin',
  'MONGODB_ADMIN_PASSWORD': 'changeme_mongo',
  'CONTENT_SECURITY_POLICY_WILDCARD': '*',
  'NOTIFICATION_TRANSPORT': 'smtp',
  'SUPER_USER_PASSWORD': 'changeme_superuser',
  'OPENCRVS_METABASE_ADMIN_EMAIL': 'admin@example.com',
  'OPENCRVS_METABASE_ADMIN_PASSWORD': 'changeme_metabase',
  'DOCKERHUB_ACCOUNT': 'opencrvs',
  'DOCKERHUB_REPO': 'ocrvs-farajaland',
  'SENDER_EMAIL_ADDRESS': 'noreply@example.com',
  'SMTP_HOST': 'smtp.example.com',
  'SMTP_PORT': '587',
  'SMTP_USERNAME': 'smtp_user',
  'SMTP_PASSWORD': 'changeme_smtp',
  'SMTP_SECURE': 'true'
}

for key, value in env_vars.items():
  os.environ[key] = value

The next command is run with Python instead of %%bash simply because %%bash doesn't support real-time output

In [118]:
inventory = 'local.macos.yml'
if linux:
    inventory = 'local.linux.yml'

cmd = f"""
mongo_user={os.environ.get('MONGODB_ADMIN_USER', '')}
mongo_pass={os.environ.get('MONGODB_ADMIN_PASSWORD', '')}
postgres_user={os.environ.get('MONGODB_ADMIN_USER', '')}
postgres_pass={os.environ.get('MONGODB_ADMIN_PASSWORD', '')}
backup_pass={os.environ.get('SUPER_USER_PASSWORD', '')}
es_pass={os.environ.get('ELASTICSEARCH_SUPERUSER_PASSWORD', '')}

ansible-playbook -i {inventory} ../server-setup/playbook.yml \\
  -e encrypted_disk_size=200g \\
  -e dockerhub_username={os.environ.get('DOCKER_USERNAME', '')} \\
  -e dockerhub_password={os.environ.get('DOCKER_PASSWORD', '')} \\
  -e mongodb_admin_username={os.environ.get('MONGODB_ADMIN_USER', '')} \\
  -e mongodb_admin_password="{os.environ.get('MONGODB_ADMIN_PASSWORD', '')}" \\
  -e backup_encryption_passphrase="{os.environ.get('SUPER_USER_PASSWORD', '')}" \\
  -e elasticsearch_superuser_password="{os.environ.get('ELASTICSEARCH_SUPERUSER_PASSWORD', '')}" \\
  -e postgres_admin_username={os.environ.get('MONGODB_ADMIN_USER', '')} \\
  -e postgres_admin_password="{os.environ.get('MONGODB_ADMIN_PASSWORD', '')}" \\
  --private-key ./.ssh/ssh-key \\
  -vv
"""

In [119]:
import subprocess

proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, shell=True, env=os.environ)
for line in proc.stdout:
    print(line, end="")
proc.wait()

ansible-playbook [core 2.18.6]
  config file = None
  configured module search path = ['/Users/riku/.ansible/plugins/modules', '/usr/share/ansible/plugins/modules']
  ansible python module location = /opt/homebrew/Cellar/ansible/11.7.0_1/libexec/lib/python3.13/site-packages/ansible
  ansible collection location = /Users/riku/.ansible/collections:/usr/share/ansible/collections
  executable location = /opt/homebrew/bin/ansible-playbook
  python version = 3.13.5 (main, Jun 11 2025, 15:36:57) [Clang 16.0.0 (clang-1600.0.26.6)] (/opt/homebrew/Cellar/ansible/11.7.0_1/libexec/bin/python)
  jinja version = 3.1.6
  libyaml = True
No config file found; using defaults
[WARNING]: Invalid characters were found in group names but not replaced, use
-vvvv to see details
redirecting (type: modules) ansible.builtin.ufw to community.general.ufw
Skipping callback 'default', as we already have a stdout callback.
Skipping callback 'minimal', as we already have a stdout callback.
Skipping callback 'oneline',

0

In [ ]:
%%bash
cd ../../
bash infrastructure/environments/update-known-hosts.sh manager.orb.local 22
bash infrastructure/environments/update-known-hosts.sh worker.orb.local 22

# Deploy OpenCRVS

After provisioning is complete, you can deploy OpenCRVS to your environment. The deployment process will:
- Download the latest OpenCRVS core docker images
- Deploy all services using Docker Swarm
- Set up databases and configurations
- Configure monitoring and alerting

Make sure to set the appropriate version tags and environment variables before deploying.

In [120]:
# Set deployment configuration
import os

# Set deployment variables
os.environ["VERSION"] = input("Enter OpenCRVS Core version (e.g., 'latest' or 'v1.7.0'): ") or "latest"
os.environ["COUNTRY_CONFIG_VERSION"] = input("Enter Country Config version (e.g., 'latest' or 'v1.0.0'): ") or "latest"
os.environ["ENVIRONMENT"] = input("Enter environment (development/staging/production): ") or "development"

# SSH configuration for local deployment
ssh_host = "manager.orb.local"  # Default for local development
ssh_user = "provision"
ssh_port = "22"
replicas = "2"

print(f"Deployment configuration:")
print(f"  Core version: {os.environ['VERSION']}")
print(f"  Country config version: {os.environ['COUNTRY_CONFIG_VERSION']}")
print(f"  Environment: {os.environ['ENVIRONMENT']}")
print(f"  SSH Host: {ssh_host}")
print(f"  SSH User: {ssh_user}")
print(f"  Replicas: {replicas}")

KeyboardInterrupt: Interrupted by user

In [121]:
os.environ["DOCKER_TOKEN"] = os.environ["DOCKER_PASSWORD"]
os.environ["COUNTRY_CONFIG_VERSION"] = "2a51ce5"
os.environ["VERSION"] = "86ac174"

os.environ["ESIGNET_REDIRECT_URL"] = "http://localhost:3000/esignet/callback"
os.environ["MOSIP_API_USERINFO_URL"] = "http://localhost:3000/esignet/callback"
os.environ["OPENID_PROVIDER_CLAIMS"] = "http://localhost:3000/esignet/callback"
os.environ["OPENID_PROVIDER_CLIENT_ID"] = "http://localhost:3000/esignet/callback"

In [124]:
%%bash
echo $COUNTRY_CONFIG_VERSION

2a51ce5


In [125]:
# Run the deployment
import subprocess
import os

# Build the deployment command
notebook_dir = os.getcwd()
ssh_key_path = os.path.join(notebook_dir, ".ssh", "ssh-key")

deploy_cmd = f"""
cd ../deployment && LC_ALL=C.UTF-8 LANG=C.UTF-8 SSH_ARGS="-i {ssh_key_path}" bash deploy.sh \\
    --host={ssh_host} \\
    --environment={os.environ['ENVIRONMENT']} \\
    --ssh_host={ssh_host} \\
    --ssh_port={ssh_port} \\
    --ssh_user={ssh_user} \\
    --version={os.environ['VERSION']} \\
    --country_config_version={os.environ['COUNTRY_CONFIG_VERSION']} \\
    --replicas={replicas}
"""

print("Starting deployment...")
print("Command to be executed:")
print(deploy_cmd)
print("\n" + "="*50 + "\n")

# Execute deployment with real-time output
proc = subprocess.Popen(
    deploy_cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    shell=True,
    env=os.environ
)

for line in proc.stdout:
    print(line, end="")

return_code = proc.wait()

if return_code == 0:
    print("\n" + "="*50)
    print("✅ Deployment completed successfully!")
    print(f"OpenCRVS should now be accessible at: https://{ssh_host}")
else:
    print("\n" + "="*50)
    print(f"❌ Deployment failed with return code: {return_code}")
    print("Check the logs above for error details.")

Starting deployment...
Command to be executed:

cd ../deployment && LC_ALL=C.UTF-8 LANG=C.UTF-8 SSH_ARGS="-i /Users/riku/Code/OpenCRVS/opencrvs-countryconfig/infrastructure/local-development/.ssh/ssh-key" bash deploy.sh \
    --host=manager.orb.local \
    --environment=production \
    --ssh_host=manager.orb.local \
    --ssh_port=22 \
    --ssh_user=provision \
    --version=86ac174 \
    --country_config_version=2a51ce5 \
    --replicas=2



bash: warning: setlocale: LC_ALL: cannot change locale (C.UTF-8): No such file or directory
/tmp/docker-compose.deps.yml /tmp/docker-compose.yml /Users/riku/Code/OpenCRVS/opencrvs-countryconfig/infrastructure/docker-compose.deploy.yml /Users/riku/Code/OpenCRVS/opencrvs-countryconfig/infrastructure/docker-compose.production-deploy.yml
Previous opencrvs version: 86ac174                      
Current opencrvs version: 86ac174
Saving redis acl
Redis acl saved to /Users/riku/Code/OpenCRVS/opencrvs-countryconfig/infrastructure/redis-acl.conf
# Host ma

In [126]:
import os
import subprocess

# Prepare environment variables for docker run
docker_env = {
  "ACTIVATE_USERS": "true",
  "GATEWAY_HOST": "https://gateway.manager.orb.local",
  "AUTH_HOST": "https://auth.manager.orb.local",
  "COUNTRY_CONFIG_HOST": "https://countryconfig.manager.orb.local",
  "SUPER_USER_PASSWORD": "changeme_superuser",
  "NODE_TLS_REJECT_UNAUTHORIZED": 0
}

core_image_tag = os.environ.get("VERSION", "latest")

docker_cmd = [
  "docker", "run",
]
for k, v in docker_env.items():
  docker_cmd += ["-e", f"{k}={v}"]
docker_cmd.append(f"ghcr.io/opencrvs/ocrvs-data-seeder:{core_image_tag}")

print("Running docker command:")
print(" ".join(docker_cmd))

proc = subprocess.Popen(
    " ".join(docker_cmd),
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    shell=True,
    env=os.environ
)

for line in proc.stdout:
    print(line, end="")

Running docker command:
docker run -e ACTIVATE_USERS=true -e GATEWAY_HOST=https://gateway.manager.orb.local -e AUTH_HOST=https://auth.manager.orb.local -e COUNTRY_CONFIG_HOST=https://countryconfig.manager.orb.local -e SUPER_USER_PASSWORD=changeme_superuser -e NODE_TLS_REJECT_UNAUTHORIZED=0 ghcr.io/opencrvs/ocrvs-data-seeder:86ac174
yarn run v1.22.22
$ NODE_OPTIONS=--dns-result-order=ipv4first ts-node -r tsconfig-paths/register src/index.ts
(node:28) Warning: Setting the NODE_TLS_REJECT_UNAUTHORIZED environment variable to '0' makes TLS connections and HTTPS requests insecure by disabling certificate verification.
(Use `node --trace-warnings ...` to show where the warning was created)
Seeding locations for v1 system
Seeding locations for v2 system (events)
Unable to seed locations for v2 events. Ensure events service is running.
{"message":"Client request error: getaddrinfo ENOTFOUND events","statusCode":502,"error":"Bad Gateway"}
Seeding users
Validation failed for roles returned from 